In [ ]:
import ase.io as ase_io
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_rmse(a, b, perc=False):
    rmse = np.sqrt(np.mean(np.square(a - b)))
    if perc:
        return 100 * rmse / b.std(ddof=1)
    return rmse

In [ ]:
def interpolate_color(start_color, end_color, num_steps):
    """
    Interpolates between two RGB colors and returns a list of colors forming a gradient.

    Parameters:
    - start_color: tuple (r, g, b) representing the RGB values of the start color
    - end_color: tuple (r, g, b) representing the RGB values of the end color
    - num_steps: integer, the number of colors in the gradient (including start and end)

    Returns:
    - list of tuples representing the interpolated RGB values.
    """
    # Create arrays of red, green, and blue values from the start and end colors
    r_values = np.linspace(start_color[0], end_color[0], num_steps)
    g_values = np.linspace(start_color[1], end_color[1], num_steps)
    b_values = np.linspace(start_color[2], end_color[2], num_steps)

    # Combine the RGB components and form the list of colors
    color_gradient = [
        (int(r), int(g), int(b)) for r, g, b in zip(r_values, g_values, b_values)
    ]

    return color_gradient


def hex_to_rgb(hex_color):
    """
    Converts a hex color string to an RGB tuple.

    Parameters:
    - hex_color: str, a hex color string (e.g., '#FF5733')

    Returns:
    - tuple of integers representing the RGB values (r, g, b)
    """
    # Remove the '#' character if present
    hex_color = hex_color.lstrip("#")

    # Convert the hex string to RGB tuple
    return tuple(int(hex_color[i : i + 2], 16) for i in (0, 2, 4))

In [ ]:
colors = ["#0000ff", "#FF0000"]
colors_rgb = [hex_to_rgb(c) for c in colors]
color_path = interpolate_color(colors_rgb[0], colors_rgb[1], 3)
color_path = [np.array(c) / 256 for c in color_path]

In [ ]:
toluene = ase_io.read("../../data/rMD17/toluene_with_atomic_energies.xyz", ":")

In [ ]:
carbon_idx = [
    0,
    1,
]  # indices of carbon atoms: 0 atom in the ring; 1: carbon atom attached to the hydroxyl group
hydro_idx = [7, 8, 9]  # indices of hydrogen atoms in the hydroxyl group
other = np.setdiff1d(
    np.arange(len(toluene[0])), carbon_idx
)  # indices of the remaining carbon atoms

In [ ]:
rmd17_forces = np.array([x.arrays["rMD17_forces"] for x in toluene])
gomace_forces = np.array([x.arrays["gomace_forces"] for x in toluene])

In [ ]:
fig = plt.figure(figsize=(5.5, 2.4), constrained_layout=True)

ax = fig.add_subplot(132)
ax.set_box_aspect(1)
ax.scatter(
    rmd17_forces[:, carbon_idx[0]],
    gomace_forces[:, carbon_idx[0]],
    s=4,
    color=color_path[0],
)
ax.scatter(
    rmd17_forces[:, carbon_idx[1]],
    gomace_forces[:, carbon_idx[1]],
    s=4,
    color=color_path[-1],
)
ax.axline((0, 0), slope=1, ls="--", color="k")
ax.set_xlabel("rMD17 forces (eV $\mathrm{\AA}^{-1}$)", fontsize=8)
ax.set_ylabel("GO-MACE-23 forces (eV $\mathrm{\AA}^{-1}$)", fontsize=8)
ax.set_xlim(-9.5, 9.5)
ax.set_ylim(-9.5, 9.5)
ax.set_yticks([-5, 0, 5])
ax.tick_params(axis="both", labelsize=8)
ax.text(
    0.02,
    0.9,
    f"RMSE={get_rmse(rmd17_forces[:, carbon_idx[0]], gomace_forces[:, carbon_idx[0]]):.1f}"
    + "eV $\mathrm{\AA}^{-1}$",
    transform=ax.transAxes,
    fontsize=8,
    color=color_path[0],
)

ax.text(
    0.3,
    0.02,
    f"RMSE={get_rmse(rmd17_forces[:, carbon_idx[1]], gomace_forces[:, carbon_idx[1]]):.1f}"
    + "eV $\mathrm{\AA}^{-1}$",
    transform=ax.transAxes,
    fontsize=8,
    color=color_path[-1],
)

labelx = -0.25
# ax.yaxis.set_label_coords(labelx, 0.5, transform=ax.transAxes)
ax.set_title("$\mathbf{b}$", fontsize=9, fontweight="bold", loc="left")

# ax.scatter(rmd17_forces[:, carbon_idx[0]] + rmd17_forces[:, carbon_idx[1]], gomace_forces[:, carbon_idx[0]] +  gomace_forces[:, carbon_idx[1]], s=4)
# ax.scatter(rmd17_forces[:, carbon_idx[1]], gomace_forces[:, carbon_idx[1]], s=4)

# ax.scatter(rmd17_forces[:, hydro_idx], gomace_forces[:, hydro_idx], s=4)

ax = fig.add_subplot(133)
ax.set_box_aspect(1)
ax.scatter(
    rmd17_forces[:, carbon_idx[0]] + rmd17_forces[:, carbon_idx[1]],
    gomace_forces[:, carbon_idx[0]] + gomace_forces[:, carbon_idx[1]],
    s=4,
    color=color_path[1],
)
ax.text(
    0.02,
    0.9,
    f"RMSE={get_rmse(rmd17_forces[:, carbon_idx[0]]+rmd17_forces[:, carbon_idx[1]], gomace_forces[:, carbon_idx[0]]+gomace_forces[:, carbon_idx[1]]):.2f}"
    + "eV $\mathrm{\AA}^{-1}$",
    transform=ax.transAxes,
    fontsize=8,
    color=color_path[1],
)

ax.axline((0, 0), slope=1, ls="--", color="k")
ax.set_xlabel("rMD17 forces (eV $\mathrm{\AA^{-1}}$)", fontsize=8)
# ax.set_ylabel("GO-MACE-23 forces (eV $\mathrm{\AA}^{-1}$)", fontsize=8)
ax.set_xlim(-9.5, 9.5)
ax.set_ylim(-9.5, 9.5)
ax.set_yticks([-5, 0, 5])
ax.set_yticklabels(())
ax.tick_params(axis="both", labelsize=8)
ax.set_title("$\mathbf{c}$", fontsize=9, fontweight="bold", loc="left")


ax = fig.add_subplot(131)
ax.set_box_aspect(1)
ax.set_title("$\mathbf{a}$", fontsize=9, fontweight="bold", loc="left")
ax.axes.set_axis_off()

# fig.savefig("./fig3.svg", dpi=300)

In [ ]:
atomic_energies = np.array([frm.arrays["gomace_atomic_energies"] for frm in toluene])

In [ ]:
fig = plt.figure(figsize=(3.5, 3), constrained_layout=True)
ax = fig.add_subplot(111)
stride = 5
ax.plot(atomic_energies[::stride, 1], color=color_path[0], lw=0.75)
ax.plot(atomic_energies[::stride, 0], color=color_path[-1], lw=0.75)
ax.plot(
    (atomic_energies[:, 0] + atomic_energies[:, 1])[::stride] / 2,
    color=color_path[1],
    lw=0.75,
)

ax.plot(atomic_energies[::stride, 2:7], "k", alpha=0.3, zorder=-1)

ax.set_ylabel("Atomic energy (eV)", fontsize=8)
ax.set_xlabel("structure index", fontsize=8)
ax.tick_params(axis="both", labelsize=8)


# fig.savefig("figS1_SI.pdf", dpi=300)